In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

data = pd.read_csv(r"C:\Users\Sanford Jone\Downloads\Bank_Personal_Loan_Modelling.csv")

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [3]:
data

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [4]:
data.corr()["Personal Loan"]

ID                   -0.024801
Age                  -0.007726
Experience           -0.007413
Income                0.502462
ZIP Code              0.000107
Family                0.061367
CCAvg                 0.366889
Education             0.136722
Mortgage              0.142095
Personal Loan         1.000000
Securities Account    0.021954
CD Account            0.316355
Online                0.006278
CreditCard            0.002802
Name: Personal Loan, dtype: float64

In [5]:
x = data.drop(['ID', 'ZIP Code', 'Personal Loan'], axis=1)
y = data['Personal Loan']

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x= scaler.fit_transform(x)

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [8]:
def fit(w):
    w = w.flatten()
    w0 = w[:66].reshape((11, 6))
    w1 = w[66:72].reshape((6,1))
    b0 = np.zeros(6)
    b1 = np.zeros(1)
    model.layers[0].set_weights([w0, b0])
    model.layers[1].set_weights([w1, b1])
    model.compile(optimizer='SGD', loss='binary_crossentropy')
    loss = model.evaluate(x_train, y_train, verbose=0)
    return loss

In [9]:
model = Sequential()
model.add(Dense(6, input_dim=x_train.shape[-1], activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Particle Swarm Optimization

In [10]:
def pso(fit, n_weights=72, max_it=50, pop_size=10, w=0.5, c1=2, c2=2):
    pos = np.random.uniform(-1, 1, (pop_size, n_weights))
    vel = np.zeros((pop_size, n_weights))
    b_pos = pos.copy()
    b_fit = np.zeros(pop_size)
    g_pos = pos[0].copy()
    g_fit = float('inf')
    prev_g_fit = 0
    
    for i in range(max_it):
        for j in range(pop_size):
            fit = fit(pos[j])
            
            if fit < b_fit[j]:
                b_fit[j] = fit
                b_pos[j] = pos[j].copy()
                
                if fit < g_fit:
                    g_fit = fit
                    g_pos = pos[j].copy()
                    
            r1 = np.random.uniform(size=n_weights)
            r2 = np.random.uniform(size=n_weights)
            vel[j] = (w * vel[j] 
                      + c1 * r1 * (b_pos[j] - pos[j])
                      + c2 * r2 * (g_pos - pos[j]))
            pos[j] += vel[j]
            
            pos[j] = np.clip(pos[j], -1, 1)
        
        if (i % 10 == 0):
            if (prev_g_fit == g_fit):
                print(f"Iter {i+1}: Best = {-g_fit:.4f} CONVERGED")
            else:
                print(f"Iter {i+1}: Best = {-g_fit:.4f}")
            prev_g_fit = g_fit
    
    return g_pos

In [ ]:
best_weights = pso(fit)
w0 = best_weights[:66].reshape((11, 6))
w1 = best_weights[66:72].reshape((6,1))
b0= np.array([0.,0.,0.,0.,0.,0.])
b1 = np.array([0.])
model.layers[0].set_weights([w0, b0])
model.layers[1].set_weights([w1, b1])

model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])

In [201]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)
accuracy_score(y_pred,y_test)

47/47 [==============================] - 0s 1ms/step


0.912